In [1]:
import pandas as pd
import numpy as np
from datetime import date
import re

pd.set_option('display.max_columns', 30)

In [2]:
def clean_unit_id(x):
    if pd.isna(x): return x
    x = str(x).strip()
    
    x = re.sub(r'(?i)(spot-?|truck|unit|#|\s)', '', x)
    
    try:
        return str(int(float(x)))
    except:
        return x

def clean_money(x):
    if pd.isna(x): return 0.0
    if isinstance(x, str):
        if '(' in x:
            x = re.sub(r'\((.*?)\)', r'-\1', x)
        
        clean_str = re.sub(r'[^\d.-]', '', x)
        
        return pd.to_numeric(clean_str, errors='coerce')
    return float(x)

In [3]:
df_finance = pd.read_excel('data/truck-finance.xlsx', sheet_name='truck_finance')

df_finance.columns = df_finance.columns.str.strip().str.lower().str.replace(' ', '_')

df_finance['clean_id'] = df_finance['unit_id'].apply(clean_unit_id)

money_cols = ['monthly_payment', 'fair_market_value', 'payoff_amount']
for col in money_cols:
    if col in df_finance.columns:
        df_finance[col] = df_finance[col].apply(clean_money)

if 'payment_end_date' in df_finance.columns:
    df_finance['payment_end_date'] = pd.to_datetime(df_finance['payment_end_date'], errors='coerce')

df_finance.head()

,id,unit_id,status,ownership_type,date_acquired,purchase_amount,monthly_payment,payment_start_date,payment_end_date,lender,make,model,year,purchase_odometer,purchase_odometer_update,odometer,odometer_update,fair_market_value,fair_market_value_date,down_payment,interest_rate,balloon_payment,clean_id
0,1,Truck 0003,INACTIVE,Owner operator owned,2025-02-01,0.0,0.0,NaN,NaT,Acza Trucking LLC,INTERNATIONAL,TT,2020,0.0,NaT,NaN,NaT,0.0,NaT,0.0,0.0,0.0,3
1,2,Truck 0008,INACTIVE,Owner operator owned,2024-01-01,0.0,0.0,NaN,NaT,Titans MVP Trucking LLC,Freightliner,TT,2016,0.0,NaT,NaN,NaT,0.0,NaT,0.0,0.0,0.0,8
2,3,Truck 0013,INACTIVE,Owner Operator Owned,11/06/2023,0.0,0.0,NaN,NaT,Prieto Trucking LLC,Western Star,TT,2007,NaN,NaT,NaN,NaT,0.0,NaT,NaN,0.0,NaN,13
3,4,Truck 0002,ACTIVE,Owner operator owned,2025-02-01,0.0,0.0,NaN,NaT,Temp Vendor,International,TT,2019,0.0,NaT,557333.0,2025-05-12 01:16:08,0.0,NaT,0.0,0.0,0.0,2
4,5,Truck 0022,INACTIVE,NaN,NaN,NaN,0.0,NaN,NaT,NaN,kenworth,TT,2006,NaN,NaT,NaN,NaT,0.0,NaT,NaN,NaN,NaN,22


In [4]:
df_maint = pd.read_excel('data/maintenancepo-truck.xlsx', sheet_name='repairs')

df_maint.columns = df_maint.columns.str.strip().str.lower().str.replace(' ', '_')

df_maint['clean_id'] = df_maint['unit_id'].apply(clean_unit_id)

if 'amount' in df_maint.columns:
    df_maint['amount'] = df_maint['amount'].apply(clean_money)


df_maint.head()

,transaction_date,unit_id,driver_name,expense_reason,description,amount,deduction_amount,pay_start,pay_end,odometer,stub_mileage,make,company_covered,clean_id
0,2024-09-27,SPOT-Truck 0005,Driver 1,Sensors & Electrical,licence plate light\n,49.02,0.0,2024-09-24,2024-09-30,598226,507237,Freightliner,49.02,5
1,2024-09-17,SPOT-Truck 0010,Driver 4,Tires,Bad steer tire and windshield wipers,722.78,0.0,2024-09-17,2024-09-23,494195,405474,Freightliner,722.78,10
2,2024-09-17,SPOT-Truck 0015,Driver 7,Tires,2 steer tires worn out,1364.51,0.0,2024-09-17,2024-09-23,193889,102679,Kenworth,1364.51,15
3,2024-09-14,SPOT-Truck 0019,Driver 10,Fluids,"WINDSHIELD WASHER FLUID -20, PETRO-CANADA DURO...",27.27,0.0,2024-09-11,2024-09-16,205702,131739,Peterbilt,27.27,19
4,2024-09-14,SPOT-Truck 0019,Driver 10,Tires,TRUCK FRONT TIRE BALANCE,79.80,0.0,2024-09-11,2024-09-16,205702,131739,Peterbilt,79.80,19


In [5]:
df_dist = pd.read_excel('data/vehicle-distance-traveled.xlsx', sheet_name='distanceTraveled')

df_dist.columns = df_dist.columns.str.strip().str.lower().str.replace(' ', '_')

df_dist['clean_id'] = df_dist['unit_id'].apply(clean_unit_id)

if 'distance' in df_dist.columns:
    df_dist['distance'] = pd.to_numeric(df_dist['distance'], errors='coerce').fillna(0)

df_dist.head()

,id,date,distance,number_of_jumps,start_odometer,end_odometer,tms_id,unit_id,clean_id
0,108010,2024-10-18,699.417683,0,NaN,NaN,10735,SPOT-Truck 0001,1
1,108011,2024-10-18,360.282090,0,NaN,NaN,11824,Truck 0006,6
2,175469,2025-01-30,284.000000,0,NaN,NaN,12153,Truck 0011,11
3,99328,2024-10-02,430.000000,0,175160.0,175590.0,10673,SPOT-Truck 0016,16
4,90574,2024-09-11,445.585144,0,NaN,NaN,11918,Truck 0020,20


In [6]:
df_paper = pd.read_excel('data/truck-paper.xlsx', sheet_name='tp_listings')

df_paper.columns = df_paper.columns.str.strip().str.lower().str.replace(' ', '_')

if 'truck_price' in df_paper.columns:
    df_paper['truck_price'] = df_paper['truck_price'].apply(clean_money)

df_paper['truck_brand'] = df_paper['truck_brand'].astype(str).str.upper().str.strip()

df_paper.head()

,id,truck_year,truck_brand,truck_model,truck_price,truck_location,truck_mileage,engine_manufacturer,truck_horsepower,transmission,transmission_manufacturer,number_of_speed,listing_date,scrape_timestamp
0,5536,2019,FREIGHTLINER,CASCADIA 126,59900.0,"3430 Irving Blvd Dallas, Texas 75247",459837,DETROIT,455,Automatic,DETROIT,12,2025-04-22 12:50:00,2025-04-22 17:56:25
1,5434,2021,FREIGHTLINER,CASCADIA 126,56900.0,"8444 Market Street Road Houston, Texas 77029",510812,DETROIT,455,Automatic,NaN,12,2025-04-22 12:36:00,2025-04-22 17:39:16
2,5271,2019,PETERBILT,567,52900.0,"1031 Cavalier Blvd Chesapeake, Virginia 23323",770726,PACCAR,510,Manual,EATON-FULLER,13,2025-04-22 12:18:00,2025-04-22 17:24:28
3,5216,2019,PETERBILT,389,104900.0,"3440 S. Main St. Harrisonburg, Virginia 22801",650747,CUMMINS,565,Manual,EATON-FULLER,18,2025-04-22 12:18:00,2025-04-22 17:20:55
4,5274,2019,PETERBILT,567,52900.0,"8900 Burge Ave Richmond, Virginia 23237",770726,PACCAR,510,Manual,EATON-FULLER,13,2025-04-22 12:18:00,2025-04-22 17:24:41


In [7]:
maint_agg = df_maint.groupby('clean_id')['amount'].sum().reset_index()
maint_agg.rename(columns={'amount': 'total_repair_cost'}, inplace=True)

dist_agg = df_dist.groupby('clean_id')['distance'].sum().reset_index()
dist_agg.rename(columns={'distance': 'total_miles_driven'}, inplace=True)

market_agg = df_paper.groupby(['truck_year', 'truck_brand'])['truck_price'].mean().reset_index()
market_agg.rename(columns={'truck_price': 'avg_market_price'}, inplace=True)

master_df = df_finance.copy()

master_df = master_df.merge(maint_agg, on='clean_id', how='left')
master_df['total_repair_cost'].fillna(0, inplace=True)

master_df = master_df.merge(dist_agg, on='clean_id', how='left')
master_df['total_miles_driven'].fillna(0, inplace=True)

master_df['make'] = master_df['make'].astype(str).str.upper().str.strip()
master_df = master_df.merge(market_agg, left_on=['year', 'make'], right_on=['truck_year', 'truck_brand'], how='left')

master_df.head()

C:\Users\ebrah\AppData\Local\Temp\ipykernel_24540\2634052715.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df['total_repair_cost'].fillna(0, inplace=True)
C:\Users\ebrah\AppData\Local\Temp\ipykernel_24540\2634052715.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


,id,unit_id,status,ownership_type,date_acquired,purchase_amount,monthly_payment,payment_start_date,payment_end_date,lender,make,model,year,purchase_odometer,purchase_odometer_update,odometer,odometer_update,fair_market_value,fair_market_value_date,down_payment,interest_rate,balloon_payment,clean_id,total_repair_cost,total_miles_driven,truck_year,truck_brand,avg_market_price
0,1,Truck 0003,INACTIVE,Owner operator owned,2025-02-01,0.0,0.0,NaN,NaT,Acza Trucking LLC,INTERNATIONAL,TT,2020,0.0,NaT,NaN,NaT,0.0,NaT,0.0,0.0,0.0,3,0.0,0.000000,2020.0,INTERNATIONAL,44044.410256
1,2,Truck 0008,INACTIVE,Owner operator owned,2024-01-01,0.0,0.0,NaN,NaT,Titans MVP Trucking LLC,FREIGHTLINER,TT,2016,0.0,NaT,NaN,NaT,0.0,NaT,0.0,0.0,0.0,8,0.0,80049.916778,2016.0,FREIGHTLINER,24464.800000
2,3,Truck 0013,INACTIVE,Owner Operator Owned,11/06/2023,0.0,0.0,NaN,NaT,Prieto Trucking LLC,WESTERN STAR,TT,2007,NaN,NaT,NaN,NaT,0.0,NaT,NaN,0.0,NaN,13,0.0,126832.850949,NaN,NaN,NaN
3,4,Truck 0002,ACTIVE,Owner operator owned,2025-02-01,0.0,0.0,NaN,NaT,Temp Vendor,INTERNATIONAL,TT,2019,0.0,NaT,557333.0,2025-05-12 01:16:08,0.0,NaT,0.0,0.0,0.0,2,0.0,41795.000000,2019.0,INTERNATIONAL,40626.481928
4,5,Truck 0022,INACTIVE,NaN,NaN,NaN,0.0,NaN,NaT,NaN,KENWORTH,TT,2006,NaN,NaT,NaN,NaT,0.0,NaT,NaN,NaN,NaN,22,0.0,0.000000,2006.0,KENWORTH,52200.000000


In [8]:
today = pd.Timestamp.now()
def calc_payoff(row):
    if pd.notnull(row['payment_end_date']) and row['payment_end_date'] > today:
        months_left = (row['payment_end_date'].year - today.year) * 12 + (row['payment_end_date'].month - today.month)
        return max(0, months_left * row['monthly_payment'])
    return 0

master_df['final_payoff'] = master_df.apply(calc_payoff, axis=1)

master_df['estimated_value'] = master_df['avg_market_price'].fillna(master_df['fair_market_value']).fillna(0)

master_df['net_equity'] = master_df['estimated_value'] - master_df['final_payoff']
master_df['cpm'] = master_df.apply(lambda x: x['total_repair_cost'] / x['total_miles_driven'] if x['total_miles_driven'] > 0 else 0, axis=1)

def make_decision(row):
    HIGH_COST_CPM = 0.20
    IDLE_MILES = 500

    if 'OWNER' in str(row['ownership_type']).upper():
        return 'IGNORE', 'Owner Asset'
    
    if (row['cpm'] > HIGH_COST_CPM) and (row['net_equity'] > 0):
        return 'SELL', 'High Cost & Positive Equity'
    
    if row['total_miles_driven'] < IDLE_MILES:
        return 'INSPECT', 'Idle Truck'
        
    if (row['cpm'] > HIGH_COST_CPM) and (row['net_equity'] <= 0):
        return 'INSPECT', 'High Cost (Negative Equity)'

    return 'KEEP', 'Good Performance'

master_df[['Recommendation', 'Reason']] = master_df.apply(make_decision, axis=1, result_type='expand')

final_cols = ['unit_id', 'Recommendation', 'Reason', 'total_repair_cost', 'total_miles_driven', 'cpm', 'net_equity']
master_df[final_cols].to_excel('Final_Report.xlsx', index=False)

master_df['Recommendation'].value_counts()

Recommendation
IGNORE     677
KEEP       243
INSPECT     44
SELL         4
Name: count, dtype: int64

In [9]:
print("🔥 TOP PRIORITY: TRUCKS TO SELL IMMEDIATELY")
sell_trucks = master_df[master_df['Recommendation'] == 'SELL']
display(sell_trucks[['unit_id', 'make', 'year', 'cpm', 'net_equity', 'Reason']])

🔥 TOP PRIORITY: TRUCKS TO SELL IMMEDIATELY


,unit_id,make,year,cpm,net_equity,Reason
222,SPOT-Truck 0406,KENWORTH,2020,0.268055,19202.803653,High Cost & Positive Equity
657,SPOT-Truck 0160,KENWORTH,2020,0.203454,16212.803653,High Cost & Positive Equity
745,SPOT-Truck 0138,PETERBILT,2022,0.330873,91463.989474,High Cost & Positive Equity
897,SPOT-Truck 0134,PETERBILT,2022,0.291360,91463.989474,High Cost & Positive Equity
